## Installing required libraries

In [23]:
!pip install d2l

## Importing required libraries

In [34]:
import os
from d2l import torch as d2l
import numpy as np
import torch
from torch import nn
import pandas as pd

## Auxiliary functions

In [37]:
def read_data_nmt():
    """Baixa e carrega o conjunto de dados inglês-francês."""
    data_dir = d2l.download_extract('fra-eng')
    with open(os.path.join(data_dir, 'fra.txt'), 'r') as f:
        return f.read()

def preprocess_nmt(text):
    """Pré-processa o texto do dataset inglês-francês."""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

def tokenize_nmt(text, num_examples=None):
    """Tokeniza o dataset inglês-francês."""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

def truncate_pad(line, num_steps, padding_token):
    """Trunca ou preenche sequências."""
    if len(line) > num_steps:
        return line[:num_steps]
    return line + [padding_token] * (num_steps - len(line))

def build_array_nmt(lines, vocab, num_steps):
    """Transforma as sequências de texto de tradução automática em minibatches."""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).sum(1)
    return array, valid_len

def load_data_nmt(batch_size, num_steps, num_examples=600):
    """Retorna o iterador e os vocabulários do conjunto de dados de tradução."""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = d2l.Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = d2l.load_array(data_arrays, batch_size)
    return data_iter, src_vocab, tgt_vocab

def vocab_size_with_different_num_examples(num_examples_list):
    src_vocab_sizes = []
    tgt_vocab_sizes = []

    for num_examples in num_examples_list:
        _, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8, num_examples=num_examples)
        src_vocab_sizes.append(len(src_vocab))
        tgt_vocab_sizes.append(len(tgt_vocab))

    return src_vocab_sizes, tgt_vocab_sizes

## Downloading raw data

In [26]:
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip', '94646ad1522d915e7b0f9296181140edcf86a4f5')

raw_text = read_data_nmt()
print(raw_text[:75])

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



## Preprocessing data

In [27]:
text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


## Tokenização

In [28]:
source, target = tokenize_nmt(text)
print(source[:6], target[:6])

[['go', '.'], ['hi', '.'], ['run', '!'], ['run', '!'], ['who', '?'], ['wow', '!']] [['va', '!'], ['salut', '!'], ['cours', '!'], ['courez', '!'], ['qui', '?'], ['ça', 'alors', '!']]


## Construção do Vocabulário

In [29]:
src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
tgt_vocab = d2l.Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])

## Obtendo o iterador de dados

In [32]:
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)

for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X)
    print('Valid lengths for X:', X_valid_len)
    print('Y:', Y)
    print('Valid lengths for Y:', Y_valid_len)
    break

X: tensor([[ 81, 180,   0,   4,   5,   5,   5,   5],
        [100, 116,   0,   4,   5,   5,   5,   5]])
Valid lengths for X: tensor([4, 4])
Y: tensor([[98, 88,  0,  4,  5,  5,  5,  5],
        [ 6,  6,  0,  4,  5,  5,  5,  5]])
Valid lengths for Y: tensor([4, 4])


## Experimento Exercício 1

In [38]:
num_examples_list = [100, 500, 1000, 5000, 10000]
src_vocab_sizes, tgt_vocab_sizes = vocab_size_with_different_num_examples(num_examples_list)

vocab_df = pd.DataFrame({
    'num_examples': num_examples_list,
    'src_vocab_size (en)': src_vocab_sizes,
    'tgt_vocab_size (fr)': tgt_vocab_sizes
})

In [39]:
vocab_df

,num_examples,src_vocab_size (en),tgt_vocab_size (fr)
0,100,40,40
1,500,159,163
2,1000,266,321
3,5000,875,1231
4,10000,1505,2252
